In [1]:
import numpy as np
import random
import pandas as pd
import math


In [2]:
# data representation of the files
# data modifications of the file system


for i in range(125):
    df = pd.read_csv(f'./finaldatasetgz/googleclusterdata{i}.csv.gz')
 # if - else clause for the copying the data   
    if i==0:
        data= df.copy()
# delete the unncessary terms in the data set
    else :
        data= pd.concat([data,df],axis=0,ignore_index=True)

    print(i)    

print(df)     

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
        Unnamed: 0     timestamp sizecpu sizeram  timeslot  gs   ds  cloud
0         47987876  2.479339e+12       s       l     688.0   1  150      5
1         47987877  2.479339e+12       s       l     688.0   1  300      3
2         47987878  2.479339e+12       s       l     688.0   1  150      0
3         47987879  2.479339e+12       s       l     688.0   1  300      3
4         47987880  2.479339e+12       s       l     688.0   2  150      8
...            ...           ...     ...     ...       ...  ..  ...    ...
386994    48374870  2.506197e+12       m       s     696.0   2  150      7
386995    

In [3]:
data

,Unnamed: 0,timestamp,sizecpu,sizeram,timeslot,gs,ds,cloud
0,0,0.000000e+00,l,l,0.0,1,150,4
1,1,0.000000e+00,l,l,0.0,2,300,8
2,2,0.000000e+00,m,l,0.0,1,150,9
3,3,0.000000e+00,m,l,0.0,2,300,0
4,4,0.000000e+00,m,l,0.0,2,300,5
...,...,...,...,...,...,...,...,...
48374870,48374870,2.506197e+12,m,s,696.0,2,150,7
48374871,48374871,2.506197e+12,m,s,696.0,2,150,0
48374872,48374872,2.506198e+12,m,s,696.0,1,150,3
48374873,48374873,2.506200e+12,m,s,696.0,1,150,6


In [4]:
class jobreq:
    #S types of Jobs serviced at each cloud, 
    # each specified by a three-tuple < ms= type of virtual machines,gs=number of virtual machines ,ds= service level aggriment>    
    def __init__(self,cloud,ms,gs,ds,jobtype):
        #request for cloud
        self.cloud = cloud
        #type of required VM there are three types small,medium, and large
        self.ms=ms
        # number  of VMs required for the job execution in general we consider 1 or 2
        self.gs=gs
        # Service level aggriment of the job  in terms of Response time 150 or 300
        self.ds=ds
        # type of jobs there are sixteen types of jobs based on the number of virtual machines 
        self.jobtype = jobtype    

In [6]:
# this function represnts the random job generation from the google cloud data structure
# 10 cloud service providers are considers and 4 cr jobs are going to execute
#  considered 16000 iterations or timeslots 

def jobgenerator(df,timeslot,clouds,possibleVMtypes,possibleDS,possiblenoofVM):
    requests = []
#     requireddata =df.loc[df['timeslot'] == timeslot+1]
# possible ds is 150 or 300
# possible vm "s","m","l"
    condition = df["timeslot"] == timeslot+1
    requireddata = df[condition].copy()
#     requireddata['cloud'] = np.random.randint(10,size = requireddata.shape[0])
# considered the ram size and rom size 
# spliting the virtual machines based on the size of ram and rom
    for i in range(len(requireddata)):
        m1=requireddata['sizeram'].iloc[i]
        m2=requireddata['sizecpu'].iloc[i]
        m = f'{m1}{m2}'
        g = requireddata['gs'].iloc[i]
        d = requireddata['ds'].iloc[i]
        c = requireddata['cloud'].iloc[i]
        t = (4*possibleVMtypes.index(m))+(2*possiblenoofVM.index(g))+possibleDS.index(d)
        requests.append(jobreq(c,m,g,d,t))
    return requests

In [7]:
# representing the cloud class and jobs........
class CSPs:
  
    def __init__(self,name,nMI,betaIT,cuspay,V,typeofServiceS=24):
        self.nMI=nMI
        self.name=name
        #all the cloud service pro  
        self.buybid={}
        #sell-bid to be submitted to auctioneer
        self.sellbid={}
         #queue maintained by cloud
        self.typeofServiceS=typeofServiceS
        self.quelen = [0 for i in range(typeofServiceS)]
        self.possibleVMtype = ['ss','sm','sl','ls','lm','ll']
        self.possibleDS = [150,300]
        self.possiblenoofVM = [1,2]
        self.V=V
        #virtual queue maintained by cloud
        self.vQue = [0 for i in range(typeofServiceS)]
        # # of jobs dropped
        self.D = [0 for i in range(typeofServiceS)]
        #cost of running the server
        self.betaIT=betaIT
#         self.cusPay = [0.02,0.02,0.02,0.04,0.04,0.04]
        self.cusPay = cuspay
        self.cMI = [30,30,30,10,10,10]
        self.allService = self.obtainServiceSdesc()
        self.maxNoofJobdrop=1000
        self.xi=cuspay
        self.pays = [0 for i in range(5)]
        self.totalpay = 0
        
        
    def obtainServiceSdesc(self):
        possibleServiceS ={}
        x=0
        for i in possibleVMtypes:
            for j in possiblenoofVM:
                for k in possibleDS:
                    possibleServiceS[x]=[i,j,k]
                    x=x+1     
        return possibleServiceS


    def initqueue(self,rsit):
        for i in rsit[self.name]:
            self.quelen[i.jobtype] = self.quelen[i.jobtype]+1
            self.updatepayoff(0,(i.gs*self.cusPay[self.possibleVMtype.index(i.ms)]))
                    
    def keyAddition(self,dicts,key,value):
        if key not in list(dicts.keys()):
            temp=[]
            temp.append(value)
            dicts[key]=temp
        else:
            dicts[key].append(value)
        return dicts
    
    
    def getBids(self):
        self.wsdashi={}
        for i in range(len(self.quelen)):        
            if self.quelen[i]==0:
                self.wsdashi = self.keyAddition(self.wsdashi,self.allService[i][0],0)
                continue        
            else:
                self.wsdashi = self.keyAddition(self.wsdashi,self.allService[i][0],(self.quelen[i]+self.vQue[i])/self.allService[i][1])
        
        self.sStarM={}      
        for key,value in self.wsdashi.items():
            index = (self.typeofServiceS/len(self.possibleVMtype))*self.possibleVMtype.index(key)+value.index(max(value))
            self.sStarM[key]=index
            self.buybid[key]=max(value)/self.V
            self.sellbid[key]=max(max(value)/self.V,self.betaIT/self.cMI[self.possibleVMtype.index(key)])
#         print("Buybid:",self.buybid) 
#         print("Sellbid:",self.sellbid)
#         print("sStar:",self.sStarM)
#         print("wsdashi:",self.wsdashi)

    
    def notosell(self,ind):
        return self.cMI[ind]*self.nMI[ind]
      
    def queueupdate(self,muSIJ,rsit):
        for services in range(len(muSIJ)):
            summation = sum(muSIJ[services][self.name])
            if summation >= self.quelen[services]:
                self.quelen[services]=0
            else:
                self.quelen[services] = self.quelen[services] - summation 
        self.initqueue(rsit)
        
    def vqueueupdate(self,muSIJ,csps):
        for services in range(len(muSIJ)):
            msInd=self.possibleVMtype.index(self.allService[services][0])
            ds = self.allService[services][2]
            gs=self.allService[services][1]
            
            if ds==150:
                epsilonS = 1000
            elif ds==300:
                epsilonS=500
            
            if self.quelen[services]==0:
                t=1
            else:
                t=0
            temp = self.vQue[services] + (1-t)*(epsilonS-sum(muSIJ[services][self.name])) - t*sum([(i.cMI[msInd]*i.nMI[msInd])/gs for i in csps])
            self.vQue[services]=max(0,temp)
            
# number of jobs droped from the cloud service providers          
    def dropjobs(self):
        for services in range(self.typeofServiceS):
            tempxi =self.xi[self.possibleVMtype.index(self.allService[services][0])]*self.allService[services][1]
            if (self.quelen[services]+self.vQue[services]) > (self.V*tempxi):
                self.D[services]=self.D[services]+self.maxNoofJobdrop
                self.quelen[services]=max(0,self.quelen[services]-self.maxNoofJobdrop)
                self.vQue[services]=max(0,self.vQue[services]-self.maxNoofJobdrop)
                self.updatepayoff(3,tempxi*1000)
# pay off after the penality of job drops 
                            
    def updatepayoff(self,ind,pay):
        self.pays[ind]= self.pays[ind]+pay
# updating the cost after droping 
        
    def provisioningcost(self,nmi):
        for keys,values in nmi.items():
            self.pays[2]= self.pays[2]+ self.betaIT*values[self.name]
            
    def totalpayment(self):
        self.totalpay = self.pays[0]+self.pays[1]-self.pays[2]-self.pays[3]-self.pays[4]
            
    def dropinend(self):
        for services in range(self.typeofServiceS):
#             print(self.quelen[services]*self.cusPay[self.possibleVMtype.index(self.allService[services][0])])
            self.pays[0]= self.pays[0]- (self.quelen[services]*self.cusPay[self.possibleVMtype.index(self.allService[services][0])]*self.allService[services][1])
    

        
        
        

In [8]:
class auctioneer:
         
    Buybid={}
    Sellbid={}
    possibleVMtype = ['ss','sm','sl','ls','lm','ll']
    
    def __init__(self,CSP):
        self.csps = CSP
        
    def allbidsandasks(self):
        for i in self.possibleVMtype:
            self.Buybid[i]={}
            self.Sellbid[i]={}
            for c in self.csps:
                self.Buybid[i][c.name]=c.buybid[i]
                self.Sellbid[i][c.name]=c.sellbid[i]
    
    def printbids(self):
        print("Buybids:",self.Buybid)
        print("SellBids:",self.Sellbid)
        
    def sortBid(self):
        for key in self.Buybid:
            self.Buybid[key]=dict(sorted(self.Buybid[key].items(),reverse=True, key=lambda item: item[1]))
            
        for key in self.Sellbid:
            self.Sellbid[key]=dict(sorted(self.Sellbid[key].items(),key=lambda item: item[1]))
            
            
    def winnerDetermination(self):
        self.winningBuyer={}
        self.winningSeller={}
        for key,value in self.Buybid.items():
            self.winningBuyer[key]={}
            s=sorted(set(list(value.values())))
            self.winningBuyer[key][list(value.items())[0][0]]=s[-2]
#         print("WinningBuyer:",self.winningBuyer)
        
        for key,value in self.Sellbid.items():
            setofwinningseller=[]
            clearingprice = list(self.winningBuyer[key].values())[0]
            clearingsellprice=0

            for keys,values in value.items():
                if values <= clearingprice:
                    setofwinningseller.append(keys)
                    clearingsellprice=values
                    
            if len(setofwinningseller) != 0:   
                setofwinningseller.pop()
            self.winningSeller[key]=[]
            self.winningSeller[key].append(setofwinningseller)
            self.winningSeller[key].append(clearingsellprice)            
#         print("WinningSeller:",self.winningSeller)
            
        
        # of VMs sold from cloud j to cloud i
        
    def determineAlpha(self):
        self.alphaMIJ = {}
        for size in self.possibleVMtype:
            self.alphaMIJ[size]=[[0 for i in range(len(self.csps))] for j in range(len(self.csps))]
            temp =[0 for x in range(len(self.csps))]
            for i in range(len(self.csps)):
                if i in self.winningSeller[size][0]:
                    temp[i]=self.csps[i].notosell(self.possibleVMtype.index(size))
            self.alphaMIJ[size][list(self.winningBuyer[size].keys())[0]]= temp.copy()
#         print("# of VMs sold j->i alphaMIJ:",self.alphaMIJ)
    
    def determineprice(self):
        for keys,values in self.winningSeller.items():
            winnername = list(self.winningBuyer[keys].keys())[0]
            temp = 0
            if len(values[0])>0:
                for i in values[0]:
                        self.csps[i].pays[1]=self.csps[i].pays[1] + self.alphaMIJ[keys][winnername][i]*values[1]
                        temp = temp + self.alphaMIJ[keys][winnername][i]
                        
            self.csps[winnername].pays[4] = self.csps[winnername].pays[4]+ temp * self.winningBuyer[keys][winnername]
                    
            
    

In [9]:
# defining the number of jobs sent from one cloud to another cloud for the execution
def determineMu(csps,auc):
    possibleService=csps[0].obtainServiceSdesc()
    
    muSIT = [[[0 for i in csps] for j in csps] for k in range(csps[0].typeofServiceS)]

    for key,value in auc.alphaMIJ.items():
        winningBuy=list(auc.winningBuyer[key].keys())
        winningSell=list(auc.winningSeller[key][0])
        servtype = int(csps[winningBuy[0]].sStarM[key])
        gs = possibleService[servtype][1]
        msInd= csps[0].possibleVMtype.index(key)
        value = [[x//gs for x in y] for y in value] 
        for i in csps:
            if i.name in winningBuy or i.name not in winningSell:
                if ((i.quelen[servtype]+i.vQue[servtype])/gs) > (i.V*i.betaIT/i.cMI[msInd]) :
                    value[i.name][i.name]=i.notosell(msInd)//gs
        muSIT[servtype]=value

    return muSIT

In [10]:
# defining the number of active service and free virtual machines 
def noOfactivatedServers(muSIT,csps,auc):
    possibleService=csps[0].obtainServiceSdesc()
    activeNMI={}
    sumAlpha={}

    for key,value in auc.alphaMIJ.items():
        temp=[0 for i in csps]
        sumAlpha[key]=[]
        for cloud in value:
            temp = [(temp[i]+cloud[i]) for i in range(len(csps))]
        sumAlpha[key]=temp.copy()    
        
    sumMuSII  = {} 
    mS = -1
    for servtype in range(len(muSIT)):
        if mS != possibleService[servtype][0]:
            mS=possibleService[servtype][0]
            sumMuSII[mS]=[]
            temp=[0 for i in csps]

        for j in range(len(muSIT[servtype])):
            temp[j] = temp[j]+muSIT[servtype][j][j]*possibleService[servtype][1]
            
        sumMuSII[mS]= temp.copy()
    
    for key,value in sumMuSII.items():
        activeNMI[key]=[(value[i]+sumAlpha[key][i])//csps[0].cMI[csps[0].possibleVMtype.index(key)] for i in range(len(csps)) ]
            
    return activeNMI       

In [11]:
# allction execution process of all the cloud service providers 
def auctionprocess(csps):
    auc = auctioneer(csps)
    auc.allbidsandasks()
    auc.sortBid()
#     auction printbids()
    auc.winnerDetermination()
    auc.determineAlpha()
    auc.determineprice()
    return auc

In [12]:
if __name__ =="__main__":
    print("    ROUND-0   \n")
# 10 cloud service providers are considerd 
    noofClouds = 10
# possible vm types are "ss","sm","sl","ls","lm","ll" based on the size of ram and rom
    possibleVMtypes = ['ss','sm','sl','ls','lm','ll']
# servicelevel aggriment
    possibleDS = [150,300]
# number of virtual machines 
    possiblenoofVM = [1,2]
#values are in $/months(730 hours)
    VMprice = [58.75,97,173.51,76.26,117.51,194] 
# per day prince is going to caliculate 
    VMpricepH = [ i/730 for i in VMprice]
    clouds = [i for i in range(noofClouds)]
    
    
    for v_value in range(6,11):
        CSP = []

        for i in range(10):
            CSP.append(CSPs(i,np.random.randint(800,1000, size=6).tolist(),random.uniform(0.3,0.4),VMpricepH,(v_value * 1000000)))

# time slots has started with -1 according the paper execution 
        timeslot = -1
        request = jobgenerator(data,timeslot,clouds,possibleVMtypes,possibleDS,possiblenoofVM)



    #resource request per round for s type of job
        rsit = [[] for i in CSP]

        for req in request: 
            rsit[req.cloud].append(req)   
# defining the queue for the each cloud service providers 
        for c in CSP:
            c.initqueue(rsit)
            c.getBids()

        auc= auctionprocess(CSP)

        muSIT = determineMu(CSP,auc)
    #     print("\n# of jobs i->j:",muSIT)
        active=noOfactivatedServers(muSIT,CSP,auc)  
    #     print("\n# of Active Servers:",active)
        for i in CSP:
            i.queueupdate(muSIT,rsit)



        timeperiod = 16000
        for t in range(timeperiod):
            print(f"\n      ROUND-{t+1}    \n ")
# for the optimzation technique we have considers 696 as optimized time slots 
            newtimeslot = t%696
            request = jobgenerator(data,newtimeslot,clouds,possibleVMtypes,possibleDS,possiblenoofVM)

            rsit = [[] for i in CSP]

            for req in request: 
                rsit[req.cloud].append(req)   
# getting buy bids after the auction execution 
            for c in CSP:
                c.getBids()
# auction is going to execute with all the cloud service providers 
            auc= auctionprocess(CSP)

            muSIT = determineMu(CSP,auc)
             print("\n# of jobs i->j:",muSIT)
            active=noOfactivatedServers(muSIT,CSP,auc)  
             print("\n# of Active Servers:",active)
    # updating the virtual queue with the incomming jobs and pending jobs
            for i in CSP:
                i.dropjobs()
                i.vqueueupdate(muSIT,CSP)
                i.queueupdate(muSIT,rsit)
                i.provisioningcost(active)
# every thousand itterations it is going to print one excel file with total jobs .total pa....
                
            if (t+1) % 1000 ==0:
                temp1={}
                temp = {}
                temp2 ={}
                temp3={}
                for i in CSP:
                    i.totalpayment()
                    temp[f"CSP{i.name}"] = i.totalpay//(t+1)
                    temp1[f"CSP{i.name}"] = i.D
                    temp2[f'CSP{i.name}'] = i.pays
                    temp3[f'CSP{i.name}'] = i.quelen

                item = pd.DataFrame.from_dict(temp, orient='index')
                item1 = pd.DataFrame.from_dict(temp1, orient='index')
                item2 = pd.DataFrame.from_dict(temp2, orient='index')
                item3 = pd.DataFrame.from_dict(temp3, orient='index')
# below excel prints the data frames with the total pays..........
                writer = pd.ExcelWriter(f'./outputpayV3/paydata{v_value}_{t+1}.xlsx', engine='xlsxwriter')

# printing the all the pays of the cloud servicr providers in excel............
                item.to_excel(writer,sheet_name="pay of all clouds")
# printing the number of jobs droped due to lose of service level aggriment......... 
                                        
                item1.to_excel(writer,sheet_name="no of job drop"
# all pays return to the cloud service provider like jon penality and buying of vitual machine buying.......
                item2.to_excel(writer,sheet_name="All Pays")
# number of pending jobs in the queue after the execution of the jobs or after itterations......
                item3.to_excel(writer,sheet_name="Number of Pending jobs in Queue")
                writer.save()

                
        
            

    
    

    ROUND-0   


      ROUND-1    
 

      ROUND-2    
 

      ROUND-3    
 

      ROUND-4    
 

      ROUND-5    
 

      ROUND-6    
 

      ROUND-7    
 

      ROUND-8    
 

      ROUND-9    
 

      ROUND-10    
 

      ROUND-11    
 

      ROUND-12    
 

      ROUND-13    
 

      ROUND-14    
 

      ROUND-15    
 

      ROUND-16    
 

      ROUND-17    
 

      ROUND-18    
 

      ROUND-19    
 

      ROUND-20    
 

      ROUND-21    
 

      ROUND-22    
 

      ROUND-23    
 

      ROUND-24    
 

      ROUND-25    
 

      ROUND-26    
 

      ROUND-27    
 

      ROUND-28    
 

      ROUND-29    
 

      ROUND-30    
 

      ROUND-31    
 

      ROUND-32    
 

      ROUND-33    
 

      ROUND-34    
 

      ROUND-35    
 

      ROUND-36    
 

      ROUND-37    
 

      ROUND-38    
 

      ROUND-39    
 

      ROUND-40    
 

      ROUND-41    
 

      ROUND-42    
 

      ROUND-43    
 

      ROUND-44    
 

      ROUND-45    
 

  


      ROUND-362    
 

      ROUND-363    
 

      ROUND-364    
 

      ROUND-365    
 

      ROUND-366    
 

      ROUND-367    
 

      ROUND-368    
 

      ROUND-369    
 

      ROUND-370    
 

      ROUND-371    
 

      ROUND-372    
 

      ROUND-373    
 

      ROUND-374    
 

      ROUND-375    
 

      ROUND-376    
 

      ROUND-377    
 

      ROUND-378    
 

      ROUND-379    
 

      ROUND-380    
 

      ROUND-381    
 

      ROUND-382    
 

      ROUND-383    
 

      ROUND-384    
 

      ROUND-385    
 

      ROUND-386    
 

      ROUND-387    
 

      ROUND-388    
 

      ROUND-389    
 

      ROUND-390    
 

      ROUND-391    
 

      ROUND-392    
 

      ROUND-393    
 

      ROUND-394    
 

      ROUND-395    
 

      ROUND-396    
 

      ROUND-397    
 

      ROUND-398    
 

      ROUND-399    
 

      ROUND-400    
 

      ROUND-401    
 

      ROUND-402    
 

      ROUND-403    
 

      ROUND-404    
 

      ROUN


      ROUND-719    
 

      ROUND-720    
 

      ROUND-721    
 

      ROUND-722    
 

      ROUND-723    
 

      ROUND-724    
 

      ROUND-725    
 

      ROUND-726    
 

      ROUND-727    
 

      ROUND-728    
 

      ROUND-729    
 

      ROUND-730    
 

      ROUND-731    
 

      ROUND-732    
 

      ROUND-733    
 

      ROUND-734    
 

      ROUND-735    
 

      ROUND-736    
 

      ROUND-737    
 

      ROUND-738    
 

      ROUND-739    
 

      ROUND-740    
 

      ROUND-741    
 

      ROUND-742    
 

      ROUND-743    
 

      ROUND-744    
 

      ROUND-745    
 

      ROUND-746    
 

      ROUND-747    
 

      ROUND-748    
 

      ROUND-749    
 

      ROUND-750    
 

      ROUND-751    
 

      ROUND-752    
 

      ROUND-753    
 

      ROUND-754    
 

      ROUND-755    
 

      ROUND-756    
 

      ROUND-757    
 

      ROUND-758    
 

      ROUND-759    
 

      ROUND-760    
 

      ROUND-761    
 

      ROUN


      ROUND-1073    
 

      ROUND-1074    
 

      ROUND-1075    
 

      ROUND-1076    
 

      ROUND-1077    
 

      ROUND-1078    
 

      ROUND-1079    
 

      ROUND-1080    
 

      ROUND-1081    
 

      ROUND-1082    
 

      ROUND-1083    
 

      ROUND-1084    
 

      ROUND-1085    
 

      ROUND-1086    
 

      ROUND-1087    
 

      ROUND-1088    
 

      ROUND-1089    
 

      ROUND-1090    
 

      ROUND-1091    
 

      ROUND-1092    
 

      ROUND-1093    
 

      ROUND-1094    
 

      ROUND-1095    
 

      ROUND-1096    
 

      ROUND-1097    
 

      ROUND-1098    
 

      ROUND-1099    
 

      ROUND-1100    
 

      ROUND-1101    
 

      ROUND-1102    
 

      ROUND-1103    
 

      ROUND-1104    
 

      ROUND-1105    
 

      ROUND-1106    
 

      ROUND-1107    
 

      ROUND-1108    
 

      ROUND-1109    
 

      ROUND-1110    
 

      ROUND-1111    
 

      ROUND-1112    
 

      ROUND-1113    
 

      ROUND-111


      ROUND-1415    
 

      ROUND-1416    
 

      ROUND-1417    
 

      ROUND-1418    
 

      ROUND-1419    
 

      ROUND-1420    
 

      ROUND-1421    
 

      ROUND-1422    
 

      ROUND-1423    
 

      ROUND-1424    
 

      ROUND-1425    
 

      ROUND-1426    
 

      ROUND-1427    
 

      ROUND-1428    
 

      ROUND-1429    
 

      ROUND-1430    
 

      ROUND-1431    
 

      ROUND-1432    
 

      ROUND-1433    
 

      ROUND-1434    
 

      ROUND-1435    
 

      ROUND-1436    
 

      ROUND-1437    
 

      ROUND-1438    
 

      ROUND-1439    
 

      ROUND-1440    
 

      ROUND-1441    
 

      ROUND-1442    
 

      ROUND-1443    
 

      ROUND-1444    
 

      ROUND-1445    
 

      ROUND-1446    
 

      ROUND-1447    
 

      ROUND-1448    
 

      ROUND-1449    
 

      ROUND-1450    
 

      ROUND-1451    
 

      ROUND-1452    
 

      ROUND-1453    
 

      ROUND-1454    
 

      ROUND-1455    
 

      ROUND-145

...


      ROUND-1569    
 

      ROUND-1570    
 

      ROUND-1571    
 

      ROUND-1572    
 

      ROUND-1573    
 

      ROUND-1574    
 

      ROUND-1575    
 

      ROUND-1576    
 

      ROUND-1577    
 

      ROUND-1578    
 

      ROUND-1579    
 

      ROUND-1580    
 

      ROUND-1581    
 

      ROUND-1582    
 

      ROUND-1583    
 

      ROUND-1584    
 

      ROUND-1585    
 

      ROUND-1586    
 

      ROUND-1587    
 

      ROUND-1588    
 

      ROUND-1589    
 

      ROUND-1590    
 

      ROUND-1591    
 

      ROUND-1592    
 

      ROUND-1593    
 

      ROUND-1594    
 

      ROUND-1595    
 

      ROUND-1596    
 

      ROUND-1597    
 

      ROUND-1598    
 

      ROUND-1599    
 

      ROUND-1600    
 

      ROUND-1601    
 

      ROUND-1602    
 

      ROUND-1603    
 

      ROUND-1604    
 

      ROUND-1605    
 

      ROUND-1606    
 

      ROUND-1607    
 

      ROUND-1608    
 

      ROUND-1609    
 

      ROUND-161


      ROUND-1911    
 

      ROUND-1912    
 

      ROUND-1913    
 

      ROUND-1914    
 

      ROUND-1915    
 

      ROUND-1916    
 

      ROUND-1917    
 

      ROUND-1918    
 

      ROUND-1919    
 

      ROUND-1920    
 

      ROUND-1921    
 

      ROUND-1922    
 

      ROUND-1923    
 

      ROUND-1924    
 

      ROUND-1925    
 

      ROUND-1926    
 

      ROUND-1927    
 

      ROUND-1928    
 

      ROUND-1929    
 

      ROUND-1930    
 

      ROUND-1931    
 

      ROUND-1932    
 

      ROUND-1933    
 

      ROUND-1934    
 

      ROUND-1935    
 

      ROUND-1936    
 

      ROUND-1937    
 

      ROUND-1938    
 

      ROUND-1939    
 

      ROUND-1940    
 

      ROUND-1941    
 

      ROUND-1942    
 

      ROUND-1943    
 

      ROUND-1944    
 

      ROUND-1945    
 

      ROUND-1946    
 

      ROUND-1947    
 

      ROUND-1948    
 

      ROUND-1949    
 

      ROUND-1950    
 

      ROUND-1951    
 

      ROUND-195


      ROUND-2253    
 

      ROUND-2254    
 

      ROUND-2255    
 

      ROUND-2256    
 

      ROUND-2257    
 

      ROUND-2258    
 

      ROUND-2259    
 

      ROUND-2260    
 

      ROUND-2261    
 

      ROUND-2262    
 

      ROUND-2263    
 

      ROUND-2264    
 

      ROUND-2265    
 

      ROUND-2266    
 

      ROUND-2267    
 

      ROUND-2268    
 

      ROUND-2269    
 

      ROUND-2270    
 

      ROUND-2271    
 

      ROUND-2272    
 

      ROUND-2273    
 

      ROUND-2274    
 

      ROUND-2275    
 

      ROUND-2276    
 

      ROUND-2277    
 

      ROUND-2278    
 

      ROUND-2279    
 

      ROUND-2280    
 

      ROUND-2281    
 

      ROUND-2282    
 

      ROUND-2283    
 

      ROUND-2284    
 

      ROUND-2285    
 

      ROUND-2286    
 

      ROUND-2287    
 

      ROUND-2288    
 

      ROUND-2289    
 

      ROUND-2290    
 

      ROUND-2291    
 

      ROUND-2292    
 

      ROUND-2293    
 

      ROUND-229


      ROUND-2595    
 

      ROUND-2596    
 

      ROUND-2597    
 

      ROUND-2598    
 

      ROUND-2599    
 

      ROUND-2600    
 

      ROUND-2601    
 

      ROUND-2602    
 

      ROUND-2603    
 

      ROUND-2604    
 

      ROUND-2605    
 

      ROUND-2606    
 

      ROUND-2607    
 

      ROUND-2608    
 

      ROUND-2609    
 

      ROUND-2610    
 

      ROUND-2611    
 

      ROUND-2612    
 

      ROUND-2613    
 

      ROUND-2614    
 

      ROUND-2615    
 

      ROUND-2616    
 

      ROUND-2617    
 

      ROUND-2618    
 

      ROUND-2619    
 

      ROUND-2620    
 

      ROUND-2621    
 

      ROUND-2622    
 

      ROUND-2623    
 

      ROUND-2624    
 

      ROUND-2625    
 

      ROUND-2626    
 

      ROUND-2627    
 

      ROUND-2628    
 

      ROUND-2629    
 

      ROUND-2630    
 

      ROUND-2631    
 

      ROUND-2632    
 

      ROUND-2633    
 

      ROUND-2634    
 

      ROUND-2635    
 

      ROUND-263


      ROUND-2937    
 

      ROUND-2938    
 

      ROUND-2939    
 

      ROUND-2940    
 

      ROUND-2941    
 

      ROUND-2942    
 

      ROUND-2943    
 

      ROUND-2944    
 

      ROUND-2945    
 

      ROUND-2946    
 

      ROUND-2947    
 

      ROUND-2948    
 

      ROUND-2949    
 

      ROUND-2950    
 

      ROUND-2951    
 

      ROUND-2952    
 

      ROUND-2953    
 

      ROUND-2954    
 

      ROUND-2955    
 

      ROUND-2956    
 

      ROUND-2957    
 

      ROUND-2958    
 

      ROUND-2959    
 

      ROUND-2960    
 

      ROUND-2961    
 

      ROUND-2962    
 

      ROUND-2963    
 

      ROUND-2964    
 

      ROUND-2965    
 

      ROUND-2966    
 

      ROUND-2967    
 

      ROUND-2968    
 

      ROUND-2969    
 

      ROUND-2970    
 

      ROUND-2971    
 

      ROUND-2972    
 

      ROUND-2973    
 

      ROUND-2974    
 

      ROUND-2975    
 

      ROUND-2976    
 

      ROUND-2977    
 

      ROUND-297